# Sistema Dinámico: Hormonas Adaptativas

**Notebook:** Sistema de actualización hormonal en tiempo real  
**Versión:** 1.0  
**Fecha:** Enero 2025

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolakorff/tfm-endocrine-llm/blob/main/examples/04_demo_dynamic.ipynb)

---

Este notebook demuestra el **sistema dinámico** que ajusta automáticamente los niveles hormonales durante la generación basándose en feedback en tiempo real.

**Contenido:**
1. Introducción al sistema dinámico
2. Generación dinámica básica
3. Visualización de trayectorias hormonales
4. Comparación estático vs dinámico
5. Análisis de cambios hormonales

## 1. Instalación

In [ ]:
!pip install -q git+https://github.com/Nicolakorff/tfm-endocrine-llm.git@v0.5.0

import endocrine_llm
print(f"Versión: {endocrine_llm.__version__}")

## 2. Imports

In [ ]:
from endocrine_llm import EndocrineModulatedLLM, HormoneProfile, HORMONE_PROFILES
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Cargar modelo
print("Cargando modelo...")
model = EndocrineModulatedLLM("distilgpt2")
print(f"Modelo listo en: {model.device}")

## 3. ¿Qué es el Sistema Dinámico?

El sistema dinámico ajusta automáticamente los niveles hormonales durante la generación:

- **Dopamina**: ↑ con alta confianza, ↓ con baja confianza
- **Cortisol**: ↑ con alta entropía (incertidumbre), ↓ con baja
- **Oxitocin**: ↑ con sentimiento positivo, ↓ con negativo
- **Serotonina**: ↓ con alta repetición, ↑ con baja
- **Adrenalina**: ↑ cuando cortisol alto + dopamina baja (estrés sin recompensa)

### Crear Perfil Dinámico

In [ ]:
# Crear perfil dinámico (todos en 0.5, aprende en tiempo real)
dynamic_profile = HormoneProfile(
    dopamine=0.5,
    cortisol=0.5,
    oxytocin=0.5,
    adrenaline=0.5,
    serotonin=0.5,
    dynamic=True,        # Activar modo dinámico
    learning_rate=0.15   # Velocidad de adaptación
)

print("Perfil dinámico creado:")
print(dynamic_profile)

## 4. Generación Dinámica Básica

In [ ]:
prompt = "I'm feeling stressed about work."

# Generar con actualización hormonal
result = model.generate_with_dynamic_hormones(
    prompt=prompt,
    initial_profile=dynamic_profile,
    max_new_tokens=50,
    update_interval=5,           # Actualizar cada 5 tokens
    return_hormone_trajectory=True
)

print("="*70)
print("GENERACIÓN DINÁMICA")
print("="*70)
print(f"\nPrompt: {prompt}")
print(f"\nTexto generado:\n{result['generated_text']}")
print(f"\nTokens generados: {result['num_tokens']}")

print("\n" + "="*70)
print("CAMBIOS HORMONALES")
print("="*70)

hormones = ['dopamine', 'cortisol', 'oxytocin', 'adrenaline', 'serotonin']
for hormone in hormones:
    initial = dynamic_profile.to_dict()[hormone]
    final = result['final_hormone_profile'][hormone]
    delta = final - initial

    arrow = "↑" if delta > 0 else "↓" if delta < 0 else "→"
    print(f"{hormone:12s}: {initial:.3f} {arrow} {final:.3f}  (Δ = {delta:+.3f})")

## 5. Visualizar Trayectoria Hormonal

In [ ]:
trajectory = result['hormone_trajectory']

# Colores para cada hormona
colors = {
    'dopamine': '#e74c3c',
    'cortisol': '#f39c12',
    'oxytocin': '#3498db',
    'adrenaline': '#9b59b6',
    'serotonin': '#2ecc71'
}

plt.figure(figsize=(14, 7))

for hormone in hormones:
    values = [step[hormone] for step in trajectory]
    steps = list(range(len(values)))

    plt.plot(steps, values,
             marker='o',
             label=hormone.capitalize(),
             color=colors[hormone],
             linewidth=2.5,
             markersize=6,
             alpha=0.8)

plt.xlabel('Update Step', fontsize=13, fontweight='bold')
plt.ylabel('Hormone Level', fontsize=13, fontweight='bold')
plt.title('Trayectoria Hormonal Durante Generación Dinámica',
          fontsize=15, fontweight='bold', pad=15)
plt.legend(loc='best', fontsize=11, framealpha=0.9)
plt.grid(True, alpha=0.3, linestyle='--')
plt.ylim([0, 1])
plt.tight_layout()
plt.show()

print(f"\n Se realizaron {len(trajectory)} actualizaciones hormonales")

## 6. Comparación: Estático vs Dinámico

In [ ]:
from endocrine_llm import TextMetrics

prompt = "I'm feeling anxious."

# Versión ESTÁTICA
static_profile = HormoneProfile(0.5, 0.5, 0.5, 0.5, 0.5, dynamic=False)
static_texts = model.generate_with_hormones(
    prompt, static_profile, max_new_tokens=50, num_return_sequences=1
)

# Versión DINÁMICA
dynamic_profile = HormoneProfile(0.5, 0.5, 0.5, 0.5, 0.5, dynamic=True, learning_rate=0.15)
dynamic_result = model.generate_with_dynamic_hormones(
    prompt, dynamic_profile, max_new_tokens=50
)

# Calcular métricas
static_metrics = TextMetrics.compute_all(static_texts[0])
dynamic_metrics = TextMetrics.compute_all(dynamic_result['generated_text'])

print("="*70)
print("COMPARACIÓN: ESTÁTICO VS DINÁMICO")
print("="*70)

print("\n[ESTÁTICO]")
print(static_texts[0])

print("\n[DINÁMICO]")
print(dynamic_result['generated_text'])

print("\n" + "="*70)
print("MÉTRICAS COMPARATIVAS")
print("="*70)

metrics_to_show = ['distinct_2', 'repetition_rate', 'sentiment_polarity']

for metric in metrics_to_show:
    s_val = static_metrics[metric]
    d_val = dynamic_metrics[metric]
    diff = d_val - s_val

    print(f"\n{metric:20s}:")
    print(f"  Estático:  {s_val:.4f}")
    print(f"  Dinámico:  {d_val:.4f}  (Δ = {diff:+.4f})")

# Cambio hormonal total
total_change = sum(
    abs(dynamic_result['final_hormone_profile'][h] - 0.5)
    for h in hormones
)
print(f"\nCambio hormonal total: {total_change:.4f}")

## 7. Probar con Diferentes Prompts

In [ ]:
test_prompts = [
    "I'm feeling happy and excited!",
    "This is very confusing and uncertain.",
    "I need help understanding this problem.",
    "Everything is going perfectly well."
]

for prompt in test_prompts:
    # Crear perfil dinámico nuevo para cada prompt
    profile = HormoneProfile(0.5, 0.5, 0.5, 0.5, 0.5, dynamic=True, learning_rate=0.15)

    result = model.generate_with_dynamic_hormones(
        prompt,
        profile,
        max_new_tokens=40,
        update_interval=5
    )

    print("\n" + "="*70)
    print(f"PROMPT: {prompt}")
    print("="*70)
    print(f"\nGeneración: {result['generated_text']}")

    # Mostrar top 3 cambios
    changes = {}
    for h in hormones:
        changes[h] = abs(result['final_hormone_profile'][h] - 0.5)

    top_changes = sorted(changes.items(), key=lambda x: -x[1])[:3]

    print("\nTop 3 cambios hormonales:")
    for hormone, change in top_changes:
        final = result['final_hormone_profile'][hormone]
        print(f"  {hormone:12s}: 0.500 → {final:.3f}  (|Δ| = {change:.3f})")

## 8. Efecto del Learning Rate

In [ ]:
prompt = "I'm feeling stressed."
learning_rates = [0.05, 0.1, 0.15, 0.2]

plt.figure(figsize=(14, 8))

for lr in learning_rates:
    profile = HormoneProfile(0.5, 0.5, 0.5, 0.5, 0.5, dynamic=True, learning_rate=lr)

    result = model.generate_with_dynamic_hormones(
        prompt,
        profile,
        max_new_tokens=50,
        update_interval=5,
        return_hormone_trajectory=True
    )

    # Calcular cambio total en cada step
    trajectory = result['hormone_trajectory']
    total_changes = []

    for step in trajectory:
        change = sum(abs(step[h] - 0.5) for h in hormones)
        total_changes.append(change)

    plt.plot(total_changes, marker='o', label=f'LR = {lr}', linewidth=2, markersize=5)

plt.xlabel('Update Step', fontsize=12, fontweight='bold')
plt.ylabel('Total Hormone Change', fontsize=12, fontweight='bold')
plt.title('Efecto del Learning Rate en Cambios Hormonales', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n Learning rates más altos → cambios más rápidos")
print("Learning rates más bajos → cambios más graduales")

## 9. Conclusión de lo hecho

- Crear perfiles hormonales dinámicos  
- Generar texto con actualización hormonal en tiempo real  
- Visualizar trayectorias hormonales  
- Comparar sistema estático vs dinámico  
- Experimentar con diferentes learning rates  

### Hallazgos

- El sistema dinámico **adapta automáticamente** las hormonas según el contenido
- Learning rates típicos: **0.05-0.20** (0.15 recomendado)
- Cambio hormonal promedio: **~0.15-0.25** por generación